In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from known.basic import Remap
from tqdm import tqdm
import torch as tt
import torch.nn as nn

import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)
warnings.filterwarnings("ignore", category=UserWarning)

default_device = 'cuda' if tt.cuda.is_available() else 'cpu'
factory = dict(device=default_device, dtype=tt.float32)

import deep, maco
from deep.rl import DQN

In [ ]:


def tenvF(): return maco.WorkersEnv.get(duration=48, n_workers=4, n_states=5, rng_seed=10293874756, max_time_ratio=1.0)
def venvF(): return maco.WorkersEnv.get(duration=48, n_workers=4, n_states=5, rng_seed=10293874756, max_time_ratio=1.0).freeze(0)
def venvsF(): return [ maco.WorkersEnv.get(duration=48, n_workers=4, n_states=5, rng_seed=10293874756, max_time_ratio=1.0).freeze(f) for f in range(5)]

env = tenvF()
def pieF(**factory): return deep.rl.S2SDQN(
    n_actions = env.A,
    embed_dim = 64,

    encoder_block_size = env.T,
    encoder_vocab_count = env.task_vocab.count,
    encoder_hidden_dim = 128,
    encoder_activation = nn.GELU(),
    encoder_num_heads = 4,
    encoder_num_layers = 1,
    encoder_norm_eps = 0.0, # final norm # keep zero to not use norm
    encoder_dropout = 0.0,
    encoder_norm_first = True,

    decoder_block_size = env.T,
    decoder_vocab_count = env.worker_vocab.count,
    decoder_hidden_dim = 128,
    decoder_activation = nn.GELU(),
    decoder_num_heads = 4,
    decoder_num_layers = 6,
    decoder_norm_eps = 0.0, # final norm # keep zero to not use norm
    decoder_dropout = 0.0,
    decoder_norm_first = True,

    dense_layer_dims = [128 ,128, 128],
    dense_actFs = [nn.Tanh(), nn.ReLU() ],
    dense_bias = True,

    xavier_init = False,

    has_target = True, 
    **factory)


In [ ]:
pie = pieF(**factory)
deep.modular.count(pie.theta)

In [ ]:
tenv = tenvF()
venvs = venvsF()

In [ ]:
epochs = 500
batch_size = 4
learn_times = 150
epsilon_range = (1.0, 0.1)

hist = DQN.train(
        # value params [T.V]
            pie                 = pie,
            pie_optA            = dict(learning_rate=0.0001, weight_decay=0.00000000, betas=(0.99, 0.999)),
            value_lrsF          = tt.optim.lr_scheduler.LinearLR, 
            value_lrsA          = dict(start_factor=1.0, end_factor=0.1, total_iters=epochs),
        # env params (training) [E]
            env                 = tenv,
            gamma               = 1.0,
            polyak              = 0.0,
        # learning params [L]
            epochs              = epochs,
            batch_size          = batch_size,
            learn_times         = learn_times,
        # explore-exploit [X]
            explore_size        = batch_size*learn_times,
            explore_seed        = None,
            epsilon_range       = epsilon_range,
            epsilon_seed        = None, 
        # memory params [M]
            memory              = None,
            memory_capacity     = 50_000, 
            memory_seed         = None, 
            min_memory          = 1_000,
        # validation params [V]
            validations_envs    = venvs, 
            validation_freq     = int(0.05*epochs), 
            validation_max_steps= None,
            validation_episodes = 1,
            validation_verbose  = 0, 
        # algorithm-specific params [A]
            double              = False,
            tuf                 = 6,
            gradient_clipping   = 0.0,
        # result params [R]
            plot_results        = True,
            save_at             = 's2sDQN',
            checkpoint_freq     = int(0.1*epochs), 
            )

In [ ]:

DQN.load_plot_training_result('s2sDQN/results.npz')